In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.examples.tutorials.mnist import input_data

### Build pipeline

In [2]:
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
dataset = tf.data.Dataset.from_tensor_slices((mnist.train.images, mnist.train.labels.astype(np.float32)))
dataset = dataset.shuffle(55000).repeat().batch(128)
val_dataset = tf.data.Dataset.from_tensor_slices((mnist.test.images, mnist.test.labels.astype(np.float32))).batch(128)

### Create a Sequential model with Keras

In [ ]:
model = tf.keras.Sequential()
# You MUST specify `input_shape`, otherwise you'll have difficulty when doing prediction!
model.add(layers.Dense(256, input_shape=(784,),activation='relu'))
model.add(layers.Dense(32, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(layers.Dense(10, activation='softmax'))

#### OR, build a model with functional composition:

In [ ]:
inputs = tf.keras.Input(shape=(784,),name='x')
# A layer instance is callable on a tensor, and returns a tensor.
x = layers.Dense(256, activation='relu')(inputs)
x = layers.Dense(32, activation='relu')(x)
predictions = layers.Dense(10, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

### Train the model 

In [ ]:
# Don't forget to specify `steps_per_epoch` when calling `fit` on a dataset.
model.fit(dataset, epochs=24, steps_per_epoch=430)

### Validate the model

In [6]:
model.evaluate(val_dataset,steps=78)

78/78 [==============================] - 0s 3ms/step


[0.08134550040062422, 0.9756610576923077]

### Use the model for prediction/inference

In [ ]:
result = model.predict(mnist.test.images)
#result = model.predict(tf.convert_to_tensor(mnist.test.images),steps=1)
predicted_digits = np.argmax(result,axis=-1)
#predicted_digits =model.predict_classes(mnist.test.images)
gt_digits =np.argmax(mnist.test.labels,axis=-1)
print("Accuracy: {}%".format(np.mean(predicted_digits==gt_digits)*100.))

### Save the keras model including parameters/variables to a single `.h5` file

In [ ]:
model.save('mnist_model.h5')
del model

### Load keras model from `.h5` file and run prediction with the model

In [3]:
model= tf.keras.models.load_model('mnist_model.h5')
model.summary()

predicted_digits =model.predict_classes(mnist.test.images[100:300])
gt_digits =np.argmax(mnist.test.labels[100:300],axis=-1)
print("Accuracy: {}%".format(np.mean(predicted_digits==gt_digits)*100.))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               200960    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8224      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                330       
Total params: 209,514
Trainable params: 209,514
Non-trainable params: 0
_________________________________________________________________
Accuracy: 99.0%


In [25]:
del model

In [4]:
predicted_digits.shape

(200,)